In [2]:
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
import timeit
from astropy.cosmology import WMAP9 as cosmo

def get_xyz(filename):
    f = fits.open(filename)
    data = f[1].data
    header = f[1].header
    ra,dec,redshift = data["RA"],data["DEC"],data["Z"]

    d = cosmo.comoving_distance(redshift)

    #plt.plot(ra,dec,".",markersize=1)
    #plt.show()

    d = d.value
    idx = (d>0) # removes only 10 galaxies
    ra,dec,d = ra[idx],dec[idx],d[idx]

    #plt.plot(ra,dec,".",markersize=1)
    #plt.show()

    conv = np.pi / 180
    x = d*np.cos(conv*ra)*np.cos(conv*dec)
    y = d*np.sin(conv*ra)*np.cos(conv*dec)
    z = d*np.sin(conv*dec)
    print(len(x))

    return (x,y,z)

x,y,z = get_xyz("ELG_N_clustering.dat.fits")
# once the sample is selected, will have 3 arrays: x,y,z

# choosing bins:
# bins = 10 ** np.linspace(np.log10(1.0/10), np.log10(20), 16)
bins = np.linspace(1.0/10, 200, 41)
bin_centers = 0.5 * (bins[1:] + bins[:-1])

t_start = timeit.default_timer() # starting timer

# calculating separation between all objects:
n = len(x) # number of objects

# DD:
sep = np.zeros(int(n*(n-1)/2)) - 1
previous = 0
for i in range(n):
    dist = np.sqrt( (x[i]-x[i+1:n])**2 + (y[i]-y[i+1:n])**2 + (z[i]-z[i+1:n])**2)
    dist = dist[dist<200]
    sep[previous:previous+len(dist)] = dist.copy()
    previous += len(dist)
print("Done with DD seps")

sep= sep[sep != -1]
dd = np.zeros(len(bins)-1)
for i in range(len(bins)-1):
    left = bins[i]
    right = bins[i+1]
    dd[i] = np.sum((sep>left) & (sep<right))

DD = dd / (n*(n-1)/2)

# averaging dr and rr over multiple random datasets:

DR,RR = np.zeros_like(DD), np.zeros_like(DD)
num_rsamples = 5
for num in range(5,10):
    randx,randy,randz = get_xyz("ELG_N_" + str(num) + "_clustering.ran.fits")
    randn = len(randx)

    # DR:
    sep = np.zeros(int(n*randn)) - 1
    previous = 0
    for i in range(n):
        dist = np.sqrt( (x[i]-randx)**2 + (y[i]-randy)**2 + (z[i]-randz)**2)
        dist = dist[dist<200]
        sep[previous:previous+len(dist)] = dist.copy()
        previous += len(dist)
    print("Done with DR seps")

    sep= sep[sep != -1]
    dr = np.zeros(len(bins)-1)
    for i in range(len(bins)-1):
        left = bins[i]
        right = bins[i+1]
        dr[i] = np.sum((sep>left) & (sep<right))
        
    DR += (dr / randn / n)


    # RR:
    sep = np.zeros(int(randn*(randn-1)/2)) - 1
    previous = 0
    for i in range(randn):
        dist = np.sqrt( (randx[i]-randx[i+1:randn])**2 + (randy[i]-randy[i+1:randn])**2 + (randz[i]-randz[i+1:randn])**2)
        dist = dist[dist<200]
        sep[previous:previous+len(dist)] = dist.copy()
        previous += len(dist)
    print("Done with RR seps")

    sep= sep[sep != -1]
    rr = np.zeros(len(bins)-1)
    for i in range(len(bins)-1):
        left = bins[i]
        right = bins[i+1]
        rr[i] = np.sum((sep>left) & (sep<right))
        
    RR += (rr / (randn*(randn-1)/2))

DR = DR / num_rsamples
RR = RR / num_rsamples

t_stop = timeit.default_timer() # stopping the timer
print("Seconds to calculate separations: ", t_stop-t_start)

# calculate Landy-Szalay estimator for 2pcf:
xi = (DD - 2*DR + RR) / RR
print(xi)

136489
Done with DD seps
209042
Done with DR seps
Done with RR seps
209836
Done with DR seps
Done with RR seps
210321
Done with DR seps
Done with RR seps
209626
Done with DR seps
Done with RR seps
210160
Done with DR seps
Done with RR seps
('Seconds to calculate separations: ', 3983.455104827881)
[ 2.73816349e+00  8.54439659e-01  4.20971719e-01  2.56978035e-01
  1.61427210e-01  1.13987919e-01  8.12221878e-02  5.99240359e-02
  4.56511200e-02  3.47222512e-02  2.74881619e-02  2.23475848e-02
  1.57520103e-02  1.23615468e-02  1.14822716e-02  7.33812726e-03
  5.66157734e-03  4.83875450e-03  4.83139937e-03  5.30239564e-03
  3.96664648e-03  2.00522744e-03  1.41714864e-03  1.85166776e-05
 -1.58426716e-03 -4.13914115e-04 -9.69563935e-04 -1.59194743e-03
 -2.81871423e-03 -2.90004199e-03 -3.25255334e-03 -2.63570954e-03
 -2.56662700e-03 -9.10282331e-04 -8.34057896e-04 -2.15617442e-03
 -1.35333312e-03 -6.37873740e-04 -8.50683128e-04  6.65594051e-04]


In [10]:
# plot results:
plt.plot(bin_centers, xi, ".")
# plt.plot(bin_centers, bin_centers**2*xi, ".")
# plt.xscale("log") # log scale
# plt.yscale("log")
plt.ylim(-0.05,0.05)
plt.xlabel(r"Separation $r$ [Mpc]", fontsize=15)
plt.ylabel(r"$\xi(r)$", fontsize=15)
plt.title("DESI ELG")
plt.show()

In [11]:
with open("corr_ELG_to200Mpc.txt", "w") as f:
    f.write("#number of galaxies: "+str(len(x))+"\n")
    for a in bin_centers:
        f.write(str(a)+" ")
    f.write("\n")
    for a in xi:
        f.write(str(a)+" ")
f.close

<function close>